In [1]:
import os
import glob
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from PIL import Image

from ultralytics import YOLO

In [2]:
# !mkdir -p data/
# !tar -xf waymo_mini.tar.gz -C data/


In [3]:
# pip install tensorflow

In [4]:
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import os

# def parse_tfrecord(example):
#     feature_description = {
#         'image/encoded': tf.io.FixedLenFeature([], tf.string),
#         'image/source_id': tf.io.FixedLenFeature([], tf.string),
#         # Add other features as needed
#     }
#     return tf.io.parse_single_example(example, feature_description)

# def save_image(image_data, output_path):
#     with open(output_path, 'wb') as f:
#         f.write(image_data)

# # Specify the path to your TFRecord file
# tfrecord_path = 'master_project/western obj test/data/Waymo_mini/waymo_format/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord'

# # Create a directory to store the extracted images
# output_dir = 'extracted_images'
# os.makedirs(output_dir, exist_ok=True)

# # Read the TFRecord file
# dataset = tf.data.TFRecordDataset(tfrecord_path)

# # Iterate over the examples in the dataset
# for i, example in enumerate(dataset):
#     parsed_example = parse_tfrecord(example)
#     image_data = parsed_example['image/encoded'].numpy()
#     source_id = parsed_example['image/source_id'].numpy().decode('utf-8')
    
#     # Save the image as a JPG file
#     output_path = os.path.join(output_dir, f'{source_id}.jpg')
#     save_image(image_data, output_path)
    
#     print(f'Saved image {i+1}: {output_path}')

In [8]:
# pip install pyarrow pandas

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pyarrow.parquet as pq
parquet_file = 'training_camera_image_10017090168044687777_6380_000_6400_000.parquet'


In [20]:
parquet_metadata = pq.read_metadata(parquet_file)

In [21]:
print("Parquet file schema:")
print(parquet_metadata.schema)

Parquet file schema:
required group field_id=-1 schema {
  optional binary field_id=-1 index (String);
  optional binary field_id=-1 key.segment_context_name (String);
  optional int64 field_id=-1 key.frame_timestamp_micros;
  optional int32 field_id=-1 key.camera_name (Int(bitWidth=8, isSigned=true));
  optional binary field_id=-1 [CameraImageComponent].image;
  optional group field_id=-1 [CameraImageComponent].pose.transform (List) {
    repeated group field_id=-1 list {
      optional double field_id=-1 item;
    }
  }
  optional float field_id=-1 [CameraImageComponent].velocity.linear_velocity.x;
  optional float field_id=-1 [CameraImageComponent].velocity.linear_velocity.y;
  optional float field_id=-1 [CameraImageComponent].velocity.linear_velocity.z;
  optional double field_id=-1 [CameraImageComponent].velocity.angular_velocity.x;
  optional double field_id=-1 [CameraImageComponent].velocity.angular_velocity.y;
  optional double field_id=-1 [CameraImageComponent].velocity.angula

In [22]:
print(f"\nNumber of rows: {parquet_metadata.num_rows}")


Number of rows: 990


In [23]:
table = pq.read_table(parquet_file)
df = table.to_pandas()

In [24]:
print("\nFirst 5 rows of data:")
print(df.head(1))


First 5 rows of data:
                                                                  key.segment_context_name  \
index                                                                                        
10017090168044687777_6380_000_6400_000;15500834...  10017090168044687777_6380_000_6400_000   

                                                    key.frame_timestamp_micros  \
index                                                                            
10017090168044687777_6380_000_6400_000;15500834...            1550083467346370   

                                                    key.camera_name  \
index                                                                 
10017090168044687777_6380_000_6400_000;15500834...                1   

                                                                         [CameraImageComponent].image  \
index                                                                                                   
10017090168044687777_63

In [26]:
model = YOLO('~/master_project/weights/yolo11m.pt')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38.8M/38.8M [00:03<00:00, 11.5MB/s]
######################################################################## 100.0%


In [27]:
first_index = df.index[0]

In [28]:
image_name = first_index.split(';')[0]

In [41]:
# print("Column names:")
# print(df.columns.tolist())
print("Row names:")
print(df.index.tolist())

Row names:
['10017090168044687777_6380_000_6400_000;1550083467346370', '10017090168044687777_6380_000_6400_000;1550083467346370', '10017090168044687777_6380_000_6400_000;1550083467346370', '10017090168044687777_6380_000_6400_000;1550083467346370', '10017090168044687777_6380_000_6400_000;1550083467346370', '10017090168044687777_6380_000_6400_000;1550083467446163', '10017090168044687777_6380_000_6400_000;1550083467446163', '10017090168044687777_6380_000_6400_000;1550083467446163', '10017090168044687777_6380_000_6400_000;1550083467446163', '10017090168044687777_6380_000_6400_000;1550083467446163', '10017090168044687777_6380_000_6400_000;1550083467545990', '10017090168044687777_6380_000_6400_000;1550083467545990', '10017090168044687777_6380_000_6400_000;1550083467545990', '10017090168044687777_6380_000_6400_000;1550083467545990', '10017090168044687777_6380_000_6400_000;1550083467545990', '10017090168044687777_6380_000_6400_000;1550083467645823', '10017090168044687777_6380_000_6400_000;1550

In [42]:
image_data_column = '10017090168044687777_6380_000_6400_000;1550083467346370'

In [43]:
for _, row in df.iterrows():
    # Get the image data from the DataFrame
    image_data = row[image_data_column].as_py()
    
    # Convert the image data to a PIL Image object
    image = Image.open(io.BytesIO(image_data))
    
    # Perform inference using the YOLO model
    results = model(image)
    
    # Process the results
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            class_label = box.cls[0].item()
            confidence = box.conf[0].item()
            
            print(f"Detected object: {class_label}, Confidence: {confidence:.2f}")
            print(f"Bounding Box: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")
            print("---")

KeyError: '10017090168044687777_6380_000_6400_000;1550083467346370'

In [31]:
# image_path = f'master_project/western obj test/data/test.jpg'

In [ ]:
# for index in df.index:
#     # Extract the image name from the index
#     image_name = index.split(';')[0]
    
#     # Construct the full image path
#     image_path = os.path.join(image_directory, f"{image_name}.jpg")
    
#     # Check if the image file exists
#     if os.path.exists(image_path):
#         # Perform inference using the YOLO model
#         results = model(image_path)
        
#         # Process the results
#         for result in results:
#             boxes = result.boxes
#             for box in boxes:
#                 x1, y1, x2, y2 = box.xyxy[0].tolist()
#                 class_label = box.cls[0].item()
#                 confidence = box.conf[0].item()
                
#                 print(f"Image: {image_name}")
#                 print(f"Detected object: {class_label}, Confidence: {confidence:.2f}")
#                 print(f"Bounding Box: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")
#                 print("---")
#     else:
#         print(f"Image file not found: {image_path}")

In [32]:
image = cv2.imread(image_path)

[ WARN:0@2775.668] global loadsave.cpp:241 findDecoder imread_('master_project/western obj test/data/test.jpg'): can't open/read file: check file path/integrity


In [25]:
# import pyarrow.parquet as pq
# import pandas as pd
# import numpy as np
# from PIL import Image
# from ultralytics import YOLO

# # Read the Parquet file into a DataFrame
# df = pq.read_table('training_camera_box_10017090168044687777_6380_000_6400_000.parquet').to_pandas()

# # Extract the image data from the DataFrame
# image_data = df['image_column_name'].values

# # Preprocess the image data
# processed_images = []
# for img_data in image_data:
#     # Convert the image data to a PIL Image object
#     img = Image.open(io.BytesIO(img_data))
    
#     # Resize the image to the desired dimensions
#     img = img.resize((640, 640))
    
#     # Convert the image to a NumPy array
#     img_array = np.array(img)
    
#     processed_images.append(img_array)

# # Convert the list of processed images to a NumPy array
# processed_images = np.array(processed_images)

# # Load the YOLO model
# model = YOLO('~/master_project/weights/yolo11m.pt')

# # Perform inference on the processed images
# results = model(processed_images)

# # Process the inference results
# for result in results:
#     # Access the detected objects, bounding boxes, and confidence scores
#     boxes = result.boxes
#     for box in boxes:
#         # Get the bounding box coordinates and class label
#         x1, y1, x2, y2 = box.xyxy[0].tolist()
#         class_label = box.cls[0].item()
#         confidence = box.conf[0].item()


## Constants

In [ ]:
%matplotlib inline

In [ ]:
WIDTH = 700

In [ ]:
HEIGHT = 500

In [ ]:
IMAGE_INDEX = 7

In [ ]:
IOU_THRESHOLD = 0.5

In [ ]:
CONF_THRESHOLD = 0.6

## Load Model

In [ ]:
model = YOLO('~/master_project/weights/yolo11m.pt')

## Load Dataset

In [ ]:
PREFIX = '../data/SSLAD-2D/labeled/train'
ANNOTATIONS_PREFIX = '../data/SSLAD-2D/labeled/annotations'

In [ ]:
files = os.listdir(PREFIX)
files[:5]

In [ ]:
# from IPython.display import Image, display

# display(Image(filename=os.path.join(PREFIX, files[0]), width=WIDTH, height=HEIGHT))

In [ ]:
annos = os.listdir(ANNOTATIONS_PREFIX)

In [ ]:
with open(os.path.join(ANNOTATIONS_PREFIX, 'instance_train.json'), 'r') as fp:
    train_annos_str = fp.read()

In [ ]:
train_annos = json.loads(train_annos_str)

In [ ]:
train_annos

## Predict

In [ ]:
test_image_path = os.path.join(PREFIX, files[IMAGE_INDEX])
test_image_path

In [ ]:
parts = test_image_path.split('_')
image_id = int(parts[2].lstrip('0'))
image_id

In [ ]:
im = Image.open(test_image_path)

In [ ]:
np_im = np.array(im)

In [ ]:
results = model(test_image_path)

In [ ]:
for box in results[0].boxes:
    print(box.xyxy.tolist()[0])
    x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
    class_id = int(box.cls)
    class_name = results[0].names[class_id]
    confidence = box.conf.item()
    
    # Draw rectangle and label on the frame
    cv2.rectangle(np_im, (x1, y1), (x2, y2), (0, 255, 0), 2)
    label = f"{class_name}: {confidence:.2f}"
    cv2.putText(np_im, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,15))
plt.imshow(np_im)

In [ ]:
im2 = Image.open(test_image_path)
np_im2 = np.array(im2)

In [ ]:
for ann in train_annos['annotations']:
    if(ann['image_id'] == image_id):
        print(ann)

        bbox = ann['bbox'] # x, y, w, h

        x1, y1, w, h = bbox

        x2, y2 = x1 + w, y1 + h
        
        cv2.rectangle(np_im2, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{class_name}"
        cv2.putText(np_im2, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,15))
plt.imshow(np_im2)